In [1]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 3.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.6/154.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 56.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [7]:
#all the imported packages./modules./etc.
import glob
import os
import cv2
import numpy as np

In [12]:
os.chdir("C://Users/abisn/OneDrive/Desktop/OpenCV/")
os.getcwd()

'C:\\Users\\abisn\\OneDrive\\Desktop\\OpenCV'

In [3]:
!pip install roboflow

In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="OTb9k6B8IEQqdAk5GpWC")
project = rf.workspace("forensicsight-ge8af").project("forensicsight-x0cje")
dataset = project.version(3).download("yolov7")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ForensicSight-3 in yolov7pytorch:: 100%|██████████| 990/990 [00:02<00:00, 371.22it/s] 


In [5]:
#Setup Directories
if not os.path.exists("./Augmentations"):
    os.mkdir("./Augmentations")
directories = ["ReflectionHorizontal", "ReflectionVertical"]
for d in directories:
    if not os.path.exists("./Augmentations/{}".format(d)):
        os.mkdir("./Augmentations/{}".format(d))
    if not os.path.exists("./Augmentations/{}/images".format(d)):
        os.mkdir("./Augmentations/{}/images".format(d))
    if not os.path.exists("./Augmentations/{}/labels".format(d)):
        os.mkdir("./Augmentations/{}/labels".format(d))
if not os.path.exists("./BoundingBoxed"):
    os.mkdir("./BoundingBoxed")
if not os.path.exists("./Augmentations/Rotation"):
    os.mkdir("./Augmentations/Rotation")
for d in ["90", "180", "270"]:
    if not os.path.exists("./Augmentations/Rotation/{}".format(d)):
        os.mkdir("./Augmentations/Rotation/{}".format(d))
    if not os.path.exists("./Augmentations/Rotation/{}/images".format(d)):
        os.mkdir("./Augmentations/Rotation/{}/images".format(d))
    if not os.path.exists("./Augmentations/Rotation/{}/labels".format(d)):
        os.mkdir("./Augmentations/Rotation/{}/labels".format(d))
if os.path.exists("./ForensicSight/train/labels"):
    os.rename("./ForensicSight/train/labels", "./ForensicSight/train/labels1")
if not os.path.exists("./ForensicSight/train/labels"):
     os.mkdir("./ForensicSight/train/labels")
inputFolder = "./ForensicSight/train/images/"

In [3]:
def write_file(arr, path, name, aug_type, sample_type):
    wString = "{} {} {} {} {}".format(sample_type, arr[0], arr[1], arr[2], arr[3])
    f = open("{}/{}_{}.txt".format(path, name, aug_type), "a")
    f.write("{}\n".format(wString))

In [9]:
def find_file(n):
    for fname in os.listdir('./Data/train/labels1/'):
        if os.path.isfile("./Data/train/labels1/{}".format(fname)):
            if n in fname:
                return fname


In [5]:
def process_points(line, width, height):
    chunks = line.split(' ')
    #print(chunks)
    #cv2.circle(image, ((int)((float(chunks[1]) * height)), (int)((float(chunks[2]) * width))), 10, (0,0,255), 10)
    #cv2.circle(image, ((int)((float(chunks[3]) * height)), (int)((float(chunks[4]) * width))), 10, (0,0,255), 10)
    #cv2.circle(image, ((int)((float(chunks[5]) * height)), (int)((float(chunks[6]) * width))), 10, (0,0,255), 10)
    #cv2.circle(image, ((int)((float(chunks[7]) * height)), (int)((float(chunks[8]) * width))), 10, (0,0,255), 10)
    pts = np.array([[(int)((float(chunks[1]) * width)),(int)((float(chunks[2]) * height))],[(int)((float(chunks[3]) * width)), (int)((float(chunks[4]) * height))],[(int)((float(chunks[5]) * width)), (int)((float(chunks[6]) * height))],[(int)((float(chunks[7]) * width)), (int)((float(chunks[8]) * height))], [(int)(chunks[0]),-1]], np.int32)
    return pts

In [6]:
import math

def rotate(origin, points, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    ar  = np.array([], np.float32)
    for point in points:
        px, py = point
        qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
        qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
        ar = np.append(ar, [qx,qy])
    return np.array([[ar[0], ar[1]], [ar[2], ar[3]], [ar[4], ar[5]], [ar[6], ar[7]]], np.int32)

In [7]:
#Horizontal Reflection Method Labels
def RH_Coords(chunks):

    relfectedChunks = chunks.copy()

    for chunk in range(len(chunks)):
        if(chunk%2 != 0 and chunk != 0):
            reflectedX = 1 - float(chunks[chunk])
            relfectedChunks[chunk] = str(reflectedX)
    
    if "\n" in relfectedChunks[len(chunks)-1]:
        relfectedChunks[-1] = relfectedChunks[-1].strip()
        #print(relfectedChunks[len(chunks)-1])


    return relfectedChunks

In [8]:
#Vertical Reflection Method Labels
def RV_Coords(chunks):
    relfectedChunks = chunks.copy()
    
    for chunk in range(len(chunks)):
        if(chunk%2 == 0 and chunk != 0):
            reflectedY = 1 - float(chunks[chunk])
            relfectedChunks[chunk] = str(reflectedY)
            
    if "\n" in relfectedChunks[len(chunks)-1]:
        relfectedChunks[-1] = relfectedChunks[-1].strip()
        #print(relfectedChunks[len(chunks)-1])

    return relfectedChunks

In [9]:
def transform(t_array, width, height):
    a  = np.array([], np.float32)
    for pair in t_array:
        for v in range(len(pair)):
            if v == 0:
                a = np.append(a, pair[0]/width)
            elif v == 1:
                a = np.append(a, pair[1]/height)
    return np.array([[a[0], a[1]], [a[2], a[3]], [a[4], a[5]], [a[6], a[7]]], np.float32)

In [10]:
def yoloTransformRotate(labelArr):
    #center = [x,y]
    # labelArray = (1, .233, .2394, .12839, 294703, .233, .23384)
    yoloCenterX = ((labelArr[0][0] + labelArr[2][0])/2)
    yoloCenterY = ((labelArr[0][1] + labelArr[2][1])/2)

    #width
    yoloWidth = abs(labelArr[0][0] - labelArr[2][0])

    #height
    yoloHeight = abs(labelArr[0][1] - labelArr[2][1])

    return np.array([yoloCenterX, yoloCenterY, yoloWidth, yoloHeight], np.float32)

In [16]:
def yoloTransform(labelArr):
    #center = [x,y]
    # labelArray = (1, .233, .2394, .12839, 294703, .233, .23384)
    point1, point2, point3, point4 = (float)(labelArr[1]), (float)(labelArr[5]), (float)(labelArr[2]), (float)(labelArr[6])
    yoloCenterX = ((point1+point2)/2)
    yoloCenterY = ((point3+point4)/2)

    #width
    yoloWidth = abs(point1-point2)

    #height
    yoloHeight = abs(point3-point4)

    return np.array([yoloCenterX, yoloCenterY, yoloWidth, yoloHeight], np.float32)

In [11]:
def RotationAugmentation(deg):
    for img in glob.glob(inputFolder + "/*.jpg"):
        image  = cv2.imread(img)
        height, width = image.shape[:2] #height is y width is x
        rotation_matrix = cv2.getRotationMatrix2D((width//2, height//2),deg,1)
        rotated_image = cv2.warpAffine(image,rotation_matrix,(width,height))
        labelFile = "./ForensicSight/train/labels1/{}".format(find_file(img[img.index("Sample")+7:img.index("_jpg")]))
        boxed_image = rotated_image
        try:
            with open(labelFile, "r") as file:
                for line in file:
                    pts = process_points(line, width, height)
                    #print(pts)
                    rotated_pts = rotate((width//2, height//2), pts, np.deg2rad(360-deg))
                    transformed_pts = transform(rotated_pts, width, height)
                    final_pts = yoloTransformRotate(transformed_pts)
                    #check_pts = np.array([[(int)((float(transformed_pts[0][0]) * width)),(int)((float(transformed_pts[0][1]) * height))],[(int)((float(transformed_pts[1][0]) * width)), (int)((float(transformed_pts[1][1]) * height))],[(int)((float(transformed_pts[2][0]) * width)), (int)((float(transformed_pts[2][1]) * height))],[(int)((float(transformed_pts[3][0]) * width)), (int)((float(transformed_pts[3][1]) * height))]], np.int32)
                    #boxed_image = cv2.polylines(boxed_image, [rotated_pts], True, (0,0,255), 10)
                    write_file(final_pts, "./Augmentations/Rotation/{}/labels/".format(deg), img[img.index("Sample"):img.index("_jpg")], "R{}".format(deg), pts[4][0])
        except FileNotFoundError:
            print(f"The file '{labelFile}' was not found.")
        except Exception as e:
            print(f"An error occurred: {str(e)}")
        cv2.imwrite("./Augmentations/Rotation/{}/images/{}_R{}.jpg".format(deg, img[img.index("Sample"):img.index("_jpg")], deg), boxed_image)
        
        

In [91]:
RotationAugmentation(270)
RotationAugmentation(180)
RotationAugmentation(90)

In [15]:
#This is augmentation for reflection horizontal
for img in glob.glob(inputFolder + "/*.jpg"):
    
    file_name = img[img.index("Sample"):img.index("_jpg")]
    labelFile = "./ForensicSight/train/labels1/{}".format(find_file(file_name))
    if not os.path.exists(labelFile):
        print(file_name)
    reflectionLabel = "./Augmentations/ReflectionHorizontal/labels/"
    image  = cv2.imread(img)
    height,width = image.shape[:2]
    flipped_image = cv2.flip(image, 1)
    boxed_image = flipped_image
    try:
        with open(labelFile, "r") as file:
            for line in file:
                chunks = line.split(' ')
                RH_arr = RH_Coords(chunks)
                final_pts = yoloTransform(RH_arr)
                #boxed_pts = np.array([[(int)((float(RH_arr[1]) * width)),(int)((float(RH_arr[2]) * height))],[(int)((float(RH_arr[3]) * width)), (int)((float(RH_arr[4]) * height))],[(int)((float(RH_arr[5]) * width)), (int)((float(RH_arr[6]) * height))],[(int)((float(RH_arr[7]) * width)), (int)((float(RH_arr[8]) * height))]], np.int32)
                #boxed_image = cv2.polylines(boxed_image, [boxed_pts], True, (0,0,255), 10)
                write_file(final_pts, reflectionLabel, file_name, "RH", chunks[0])
                
    except FileNotFoundError:
        print(f"The file '{labelFile}' was not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    cv2.imwrite("./Augmentations/ReflectionHorizontal/images/{}_RH.jpg".format(img[img.index("Sample"):img.index("_jpg")]), boxed_image)


An error occurred: name 'yoloTransform' is not defined
An error occurred: name 'yoloTransform' is not defined
An error occurred: name 'yoloTransform' is not defined
An error occurred: name 'yoloTransform' is not defined
An error occurred: name 'yoloTransform' is not defined
An error occurred: name 'yoloTransform' is not defined
An error occurred: name 'yoloTransform' is not defined
An error occurred: name 'yoloTransform' is not defined
An error occurred: name 'yoloTransform' is not defined


KeyboardInterrupt: 

In [93]:
#This is augmentation for reflection vertical
for img in glob.glob(inputFolder + "/*.jpg"):
    file_name = img[img.index("Sample"):img.index("_jpg")]
    labelFile = "./ForensicSight/train/labels1/{}".format(find_file(file_name))
    reflectionLabel = "./Augmentations/ReflectionVertical/labels/"
    image  = cv2.imread(img)
    height,width = image.shape[:2]
    vert_image = cv2.flip(image, 0)
    boxed_image = vert_image
    try:
        with open(labelFile, "r") as file:
            for line in file:
                chunks = line.split(' ')
                RV_arr = RV_Coords(chunks)
                final_pts = yoloTransform(RV_arr)
                #boxed_pts = np.array([[(int)((float(RV_arr[1]) * width)),(int)((float(RV_arr[2]) * height))],[(int)((float(RV_arr[3]) * width)), (int)((float(RV_arr[4]) * height))],[(int)((float(RV_arr[5]) * width)), (int)((float(RV_arr[6]) * height))],[(int)((float(RV_arr[7]) * width)), (int)((float(RV_arr[8]) * height))]], np.int32)
                #boxed_image = cv2.polylines(boxed_image, [boxed_pts], True, (0,0,255), 10)
                write_file(final_pts, reflectionLabel, file_name, "RV", chunks[0])

    except FileNotFoundError:
        print(f"The file '{labelFile}' was not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    cv2.imwrite("./Augmentations/ReflectionVertical/images/{}_RV.jpg".format(img[img.index("Sample"):img.index("_jpg")]), boxed_image)
    

In [17]:
#this is transformation of the original points
for img in glob.glob(inputFolder + "/*.jpg"):
    file_name = img[img.index("Sample"):img.index("_jpg")]
    labelFile = "./ForensicSight/train/labels1/{}".format(find_file(file_name))
    OGLabel = "./ForensicSight/train/labels"
    try:
        with open(labelFile, "r") as file:
            for line in file:
                chunks = line.split(' ')
                final_pts = yoloTransform(chunks)
                wString = "{} {} {} {} {}".format(chunks[0], final_pts[0], final_pts[1], final_pts[2], final_pts[3])
                f = open("{}/{}.txt".format(OGLabel, img[img.index("Sample"):len(img)-4]), "a")
                f.write("{}\n".format(wString))
    except FileNotFoundError:
        print(f"The file '{labelFile}' was not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [19]:
folderPath = "./Augmentations/ReflectionHorizontal/images"
for img in glob.glob(folderPath + "/*.jpg"):
    print(img[img.index("Sample"):img.index("_jpg")])

ValueError: substring not found

In [4]:
#Data Validation for Augmentation folders
validate = ["Rotation/90", "Rotation/180", "Rotation/270", "ReflectionHorizontal", "ReflectionVertical"]
for v in validate:
    print("{}: {}".format(v,len(os.listdir("./Augmentations/{}/images/".format(v)))))
    print("{}: {}".format(v,len(os.listdir("./Augmentations/{}/labels/".format(v)))))
print("{}: {}".format("BoundingBoxed",len(os.listdir("./BoundingBoxed/"))))

Rotation/90: 391
Rotation/90: 390
Rotation/180: 391
Rotation/180: 390
Rotation/270: 391
Rotation/270: 390
ReflectionHorizontal: 391
ReflectionHorizontal: 390
ReflectionVertical: 391
ReflectionVertical: 390
BoundingBoxed: 391


In [14]:
#Data Validation for Original Images & Labels
counterImg = 0
inputFolder = './ForensicSight/train/images/'
for img in glob.glob(inputFolder + "/*.jpg"):
    counterImg += 1
print('Image:', counterImg)

counterLab = 0
inputFolder = './ForensicSight/train/labels1/'
for lbl in glob.glob(inputFolder + "/*.txt"):
    counterLab += 1
print('Labels:', counterLab)

Image: 391
Labels: 391


In [162]:
#function to draw a bounding box from points file
for img in glob.glob(inputFolder + "./*.jpg"):
    #print(img)
    image  = cv2.imread(img)
    h,w = image.shape[:2]
    d = {}
    labelFile = "./ForensicSight/train/labels1/{}".format(find_file(img[img.index("Sample")+7:img.index("_jpg")]))
    boxed_image = image
    try:
        with open(labelFile, "r") as file:
            for line in file:
                pts = process_points(line, w, h)
                boxed_image = cv2.polylines(boxed_image, [pts[0:4]], True, (0,0,255), 10)
    except FileNotFoundError:
        print(f"The file '{labelFile}' was not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    cv2.imwrite("./BoundingBoxed/{}_BB.jpg".format(img[img.index("Sample"):img.index("_jpg")]), boxed_image)
    

./ForensicSight/train/images/.\Sample_107A_jpg.rf.8c0feadb8787b028c065557c91376a6f.jpg
./ForensicSight/train/images/.\Sample_107B_jpg.rf.b86ec60522c2fc31c4568d83a3707c04.jpg
./ForensicSight/train/images/.\Sample_120A_jpg.rf.86f20835ecf97d27b62a933befd77db2.jpg
./ForensicSight/train/images/.\Sample_120B_jpg.rf.e8f0fc1e353b581d12ac6dea5c145bfb.jpg
./ForensicSight/train/images/.\Sample_120C_jpg.rf.c8fb9fbcf5a2aba4f5b536682ff13b56.jpg
./ForensicSight/train/images/.\Sample_120D_jpg.rf.d08ca2f76fa2256123a0aec18cb8bb2a.jpg
./ForensicSight/train/images/.\Sample_120E_jpg.rf.6316e18e08908e72df7bf18f2b07d23f.jpg
./ForensicSight/train/images/.\Sample_120F_jpg.rf.4bf9080179ab5800ea617556fa7c1891.jpg
./ForensicSight/train/images/.\Sample_131A_jpg.rf.79b46defcf6dee68ee34e9ae04d27645.jpg
./ForensicSight/train/images/.\Sample_131B_jpg.rf.c3e48b91fbc16944d997b55d243e2561.jpg
./ForensicSight/train/images/.\Sample_132A_jpg.rf.8757d94ed30e4b997a33eb7109315bb9.jpg
./ForensicSight/train/images/.\Sample_132B_

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9a7a2f5b-949c-491a-a647-833490711560' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>

In [30]:
# MAKES IMAGES SQUARES

# h - height
# w - width
# coor - absolute position

from PIL import Image, ImageOps
import glob
import cv2

def ImgResizing():
    inputFolder = "./Data/train/images/"
    outputFolder = "./Data/train/sqImages"

    # Create the output folder if it doesn't exist
    os.makedirs(outputFolder, exist_ok=True)

    for img_path in glob.glob(inputFolder + "*.jpg"):
        # Open the image using PIL (since we are using PIL functions later)
        
        img = Image.open(img_path)

        # Get the size of the original image
        w, h = img.size

        if h < w:
            addedSize = (w - h) // 2
            border = (0, addedSize, 0, addedSize)  # Add space at the top and bottom
        elif w < h:
            addedSize = (h - w) // 2
            border = (addedSize, 0, addedSize, 0)  # Add space at the left and right

        # Create a new image with the added border
        new_image = ImageOps.expand(img, border=border, fill='black')

        base_name = os.path.basename(img_path)
        name, ext = os.path.splitext(base_name)
        new_name = f"{name}Sq{ext}"



        # Create the full path for the new image
        save_path = os.path.join(outputFolder, new_name)

        # Save the new image
        print(save_path)
        new_image.save(save_path)
        #new_image.show()

        # Remove this if you want to process all images

ImgResizing()


./Data/train/sqImages/Sample_107A_jpg.rf.8c0feadb8787b028c065557c91376a6fSq.jpg
./Data/train/sqImages/Sample_107B_jpg.rf.b86ec60522c2fc31c4568d83a3707c04Sq.jpg
./Data/train/sqImages/Sample_120A_jpg.rf.86f20835ecf97d27b62a933befd77db2Sq.jpg
./Data/train/sqImages/Sample_120B_jpg.rf.e8f0fc1e353b581d12ac6dea5c145bfbSq.jpg
./Data/train/sqImages/Sample_120C_jpg.rf.c8fb9fbcf5a2aba4f5b536682ff13b56Sq.jpg
./Data/train/sqImages/Sample_120D_jpg.rf.d08ca2f76fa2256123a0aec18cb8bb2aSq.jpg
./Data/train/sqImages/Sample_120E_jpg.rf.6316e18e08908e72df7bf18f2b07d23fSq.jpg
./Data/train/sqImages/Sample_120F_jpg.rf.4bf9080179ab5800ea617556fa7c1891Sq.jpg
./Data/train/sqImages/Sample_131A_jpg.rf.79b46defcf6dee68ee34e9ae04d27645Sq.jpg
./Data/train/sqImages/Sample_131B_jpg.rf.c3e48b91fbc16944d997b55d243e2561Sq.jpg
./Data/train/sqImages/Sample_132A_jpg.rf.8757d94ed30e4b997a33eb7109315bb9Sq.jpg
./Data/train/sqImages/Sample_132B_jpg.rf.aa5abef4534e51196d28e6464355e093Sq.jpg
./Data/train/sqImages/Sample_132C_jpg.rf

In [31]:
all_items = os.listdir("./Data/train/sqImages")
print(all_items)

['Sample_107A_jpg.rf.8c0feadb8787b028c065557c91376a6fSq.jpg', 'Sample_107B_jpg.rf.b86ec60522c2fc31c4568d83a3707c04Sq.jpg', 'Sample_120A_jpg.rf.86f20835ecf97d27b62a933befd77db2Sq.jpg', 'Sample_120B_jpg.rf.e8f0fc1e353b581d12ac6dea5c145bfbSq.jpg', 'Sample_120C_jpg.rf.c8fb9fbcf5a2aba4f5b536682ff13b56Sq.jpg', 'Sample_120D_jpg.rf.d08ca2f76fa2256123a0aec18cb8bb2aSq.jpg', 'Sample_120E_jpg.rf.6316e18e08908e72df7bf18f2b07d23fSq.jpg', 'Sample_120F_jpg.rf.4bf9080179ab5800ea617556fa7c1891Sq.jpg', 'Sample_131A_jpg.rf.79b46defcf6dee68ee34e9ae04d27645Sq.jpg', 'Sample_131B_jpg.rf.c3e48b91fbc16944d997b55d243e2561Sq.jpg', 'Sample_132A_jpg.rf.8757d94ed30e4b997a33eb7109315bb9Sq.jpg', 'Sample_132B_jpg.rf.aa5abef4534e51196d28e6464355e093Sq.jpg', 'Sample_132C_jpg.rf.f1fba6c1d14ec4fb5141e9d533d274a9Sq.jpg', 'Sample_132D_jpg.rf.56784570d2e4a37c35442a4a55816102Sq.jpg', 'Sample_132_Label_jpg.rf.c191d59a9e06c8d45f03cf245d74f999Sq.jpg', 'Sample_134A_jpg.rf.ba3b43b046104a4c54642a72376e2e4fSq.jpg', 'Sample_134B_jpg.r

In [10]:


def LabelResizing():
    inputFolder = './Data/train/images/'
    outputFolder = './Data/train/sqLabels/'

    # Create the output folder if it doesn't exist
    os.makedirs(outputFolder, exist_ok=True)

    for img_path in glob.glob(inputFolder + "*.jpg"):
        file_name = os.path.basename(img_path)
        base_name, _ = os.path.splitext(file_name)
        labelFile = "./Data/train/labels1/{}".format(find_file(base_name))
        new_label_path = os.path.join(outputFolder, base_name + "_sq.txt")

        img = Image.open(img_path)
        w, h = img.size
        if h < w:
            addedSize = (w - h) // 2
            addedDecimal = addedSize / w
            taller = False
        elif w < h:
            addedSize = (h - w) // 2
            addedDecimal = addedSize / h
            taller = True

        try:
            with open(labelFile, "r") as file, open(new_label_path, "w") as new_file:
                for line in file:
                    newChunks = []
                    chunks = line.split(' ')
                    newChunks.append(chunks[0])
                    for chunk in range(1, len(chunks)):
                        coor = float(chunks[chunk])
                        if taller and chunk % 2 != 0:
                            newCoor = coor + addedDecimal
                        elif not taller and chunk % 2 == 0:
                            newCoor = coor + addedDecimal
                        else:
                            newCoor = coor
                        newChunks.append(str(newCoor))
                    new_file.write(' '.join(newChunks) + '\n')

        except FileNotFoundError:
            print(f"The file '{labelFile}' was not found.")
        except Exception as e:
            print(f"An error occurred: {str(e)}")



LabelResizing()

In [5]:
img_path = '/root/ForensicSight/Data/train/images/Sample_107A_jpg.rf.8c0feadb8787b028c065557c91376a6f.jpg'

img = Image.open(img_path)

w, h = img.size

print( h, " ", w)

3201   3648


In [4]:
from PIL import Image, ImageOps
import glob
import cv2